In [2]:
import requests
import base64
import numpy as np
from PIL import Image

In [22]:
with open('granny/timber_wolf.png', 'rb') as f:
    input_data = base64.b64encode(f.read()).decode()


# input_image = Image.open('granny/granny_8.jpeg')
# input_data = array_to_base64(np.array(input_image))

def query(input_data):
    response = requests.post('http://granny.advml.com/score', json={'data': input_data})
    return response.json()


query(input_data)['output'][0:5]

[[0.28575703501701355, 'timber wolf'],
 [0.13912102580070496, 'red wolf'],
 [0.1217745989561081, 'coyote'],
 [0.02413186803460121, 'dingo'],
 [0.01603207364678383, 'Eskimo dog']]

In [25]:
from PIL import Image
from io import BytesIO

def array_to_base64(arr):
    img = Image.fromarray(arr)
    with BytesIO() as buffer:
        img.save(buffer, 'png') # must be png to achive same score
        return base64.b64encode(buffer.getvalue()).decode()

In [26]:

# testing the base64 encoding fuction
img_test = Image.open('granny/timber_wolf.png')

img_arr = np.array(img_test)

img_input = array_to_base64(img_arr)
query(img_input)['output'][0:5]

[[0.28575703501701355, 'timber wolf'],
 [0.13912102580070496, 'red wolf'],
 [0.1217745989561081, 'coyote'],
 [0.02413186803460121, 'dingo'],
 [0.01603207364678383, 'Eskimo dog']]

In [ ]:
def get_diff(s1, s2):
    return np.linalg.norm(s1 - s2, axis=(1, 2))

def orthog_perturb(delta, prev, target):
    perturb = np.random.randn(1, 224, 224, 3)
    perturb /= np.linalg.norm(perturb, axis=(1, 2))
    perturb *= delta * np.mean(get_diff(target, prev))
    
    # Projection
    diff = target - prev
    diff /= get_diff(target, prev)
    
    perturb -= (np.vdot(perturb, diff) / np.linalg.norm(diff) ** 2) * diff
    
    overflow = prev + perturb
    perturb -= overflow * (overflow > 1)
    
    underflow = prev + perturb
    perturb -= underflow * (underflow < 0)
    
    return perturb

def forward_perturb(epsilon, prev, target):
    return (target - prev) * epsilon

def predict(sample):
    img = T.ToPILImage()(torch.Tensor(sample).moveaxis(3, 1).squeeze())
    img_bytes = img.tobytes()
    b64_img_bytes = base64.urlsafe_b64encode(img_bytes)
    
    while True:
        try:
            r = requests.post("https://hotdog.fly.dev/score", data=json.dumps({"input": b64_img_bytes.decode()}))

            if len(r.text) > 120:
                return 1
            else:
                return 0
        except:
            pass

def show_image(img):
    image_tensor = torch.Tensor(img).moveaxis(3, 1)
    plt.imshow(image_tensor.squeeze().permute(1, 2, 0))
    
chester = Im.open('../input/ctf-data/ctf-data/chester.png')
chester = chester.resize((224, 224))
chester = TF.to_tensor(chester)
chester = chester.unsqueeze(0)

hotdog = Im.open('../input/ctf-data/ctf-data/hotdog.png')
hotdog = hotdog.resize((224, 224))
hotdog = TF.to_tensor(hotdog)
hotdog = hotdog.unsqueeze(0)

initial_sample = torch.moveaxis(hotdog, 1, 3).numpy()
target_sample = torch.moveaxis(chester, 1, 3).numpy()

adversarial_sample = initial_sample

epsilon = 0.1
delta = 0.01

ATTACK_CLASS = 1

saved_images = []
saved_queries = []

#################################################################################################
#################################################################################################

# The following is the code to run the actual attack algorithm. Because the servers are no longer
# online, it will not run because it makes queries to the server to determine whether an image
# is adversarial or not. I've included the final image I used to obtain the flag, and was a
# direct result of running the below algorithm. There were a few small tweaks I made to the
# original starting image - I believe I rotated the hotdog to be more inline with the Chester,
# but this was simply to help the algorithm converge faster.
#
# Note - the full process took a few hours to complete, mostly because querying the flag server
# is slow.

#################################################################################################
#################################################################################################

# Move towards the boundary
# queries = 0
# while True:
#     trial_sample = adversarial_sample + forward_perturb(epsilon, adversarial_sample, target_sample)
#     prediction = predict(trial_sample)

#     queries += 1
#     if prediction == ATTACK_CLASS:
#         adversarial_sample = trial_sample
#         break
#     else:
#         epsilon *= 0.9
        
# # Run iterative attack
# for j in range(100000):
#     if j % 10 == 0:
#         print(f'Queries: {queries}')
#         print(f'MSE: {np.mean(get_diff(target_sample, adversarial_sample))}')
#         saved_images.append(adversarial_sample)
#         saved_queries.append(queries)
        
#     while True:
#         d_score = 0
#         good_sample = np.zeros(0)
        
#         for i in range(10):
#             trial_sample = adversarial_sample + orthog_perturb(delta, adversarial_sample, target_sample)
#             prediction = predict(trial_sample)
#             queries += 1
            
#             if prediction == ATTACK_CLASS:
#                 good_sample = trial_sample
#                 d_score += 0.1

#         if d_score > 0.0:
#             if d_score < 0.3:
#                 delta *= 0.9
#             elif d_score > 0.7:
#                 delta /= 0.9

#             adversarial_sample = good_sample
#             break
#         else:
#             delta *= 0.9
            
#     # Forward step
#     step = 0
#     while True:
#         step += 1
#         trial_sample = adversarial_sample + forward_perturb(epsilon, adversarial_sample, target_sample)
#         prediction = predict(trial_sample)
#         queries += 1
        
#         if prediction == ATTACK_CLASS:
#             adversarial_sample = trial_sample
#             epsilon /= 0.5
#             break
#         elif step > 500:
#             break
#         else:
#             epsilon *= 0.5

# adversarial = Im.open('../input/ctf-data/ctf-data/adversarial-chester.png')
# adversarial = adversarial.resize((224, 224))

# Guess it's close enough to fool the classifier!
# plt.imshow(adversarial)